# Hausarbeit - Reporting und Visualisierung
### Einflussfaktoren auf die Schulleistungen im Rahmen der Oberschule
#### MADS 2023oB

## Environment setup

In [ ]:
%%capture
# !pip install numpy pandas plotly
# or
# !. venv/bin/activate

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data transformations

### Data description

**Attributes for both student-mat.csv (Math course) and student-por.csv (Portuguese language course) datasets:**

1 school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)

2 sex - student's sex (binary: 'F' - female or 'M' - male)

3 age - student's age (numeric: from 15 to 22)

4 address - student's home address type (binary: 'U' - urban or 'R' - rural)

5 famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)

6 Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)

7 Medu - mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – 
secondary education or 4 – higher education)

8 Fedu - father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – 
secondary education or 4 – higher education)

9 Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or 
police), 'at_home' or 'other')

10 Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or 
police), 'at_home' or 'other')

11 reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 
'other')

12 guardian - student's guardian (nominal: 'mother', 'father' or 'other')

13 traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 
4 - >1 hour)

14 studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)

15 failures - number of past class failures (numeric: n if 1<=n<3, else 4)

16 schoolsup - extra educational support (binary: yes or no)

17 famsup - family educational support (binary: yes or no)

18 paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)

19 activities - extra-curricular activities (binary: yes or no)

20 nursery - attended nursery school (binary: yes or no)

21 higher - wants to take higher education (binary: yes or no)

22 internet - Internet access at home (binary: yes or no)

23 romantic - with a romantic relationship (binary: yes or no)

24 famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)

25 freetime - free time after school (numeric: from 1 - very low to 5 - very high)

26 goout - going out with friends (numeric: from 1 - very low to 5 - very high)

27 Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)

28 Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)

29 health - current health status (numeric: from 1 - very bad to 5 - very good)

30 absences - number of school absences (numeric: from 0 to 93)

**these grades are related with the course subject, Math or Portuguese:**

31 G1 - first period grade (numeric: from 0 to 20)

31 G2 - second period grade (numeric: from 0 to 20)

32 G3 - final grade (numeric: from 0 to 20, output target)

Additional note: there are several (382) students that belong to both datasets . 
These students can be identified by searching for identical attributes
that characterize each student, as shown in the annexed R file.

### Data loading and examination


In [ ]:
mat_df = pd.read_csv('data/student-mat.csv', sep=';')
por_df = pd.read_csv('data/student-por.csv', sep=';')

info_df = pd.DataFrame([['mat_df', len(mat_df), len(mat_df.columns), np.mean(mat_df.isnull().sum())],
                        ['por_df', len(por_df), len(por_df.columns), np.mean(por_df.isnull().sum())]], 
                        columns=['table name', 'table size', 'number of columns', 'average missing values per column'])
info_df

In [ ]:
print(f'mat_df columns: \n\n {mat_df.columns} \n')
print(f'por_df columns: \n\n {por_df.columns} \n')

In [ ]:
mat_df.head()

In [ ]:
por_df.head()

In [ ]:
px.imshow(mat_df.corr(numeric_only=True))
px.imshow(por_df.corr(numeric_only=True))

### Data merging

In [ ]:
merging_colums = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','nursery','internet']

In [ ]:
left_mixed_df = pd.merge(por_df,mat_df,'left', on=merging_colums)
# left_mixed_df.to_csv('data/student-merge-left.csv')

In [ ]:
inner_mixed_df = pd.merge(por_df,mat_df,'inner', on=merging_colums)
# inner_mixed_df.to_csv('data/student-merge-inner.csv')

In [ ]:
def check_contraditories(df:pd.DataFrame, columns='all'):
    # This cell checks for contraditory values within the merged dataset
    double_columns = [x for x in list(por_df.columns) if x not in merging_colums]
    check_columns = columns if columns != 'all' else double_columns # 'guardian', 'romantic', 'famrel', 'goout', 'health', 'Dalc', 'Walc', 'higher'
    df = pd.DataFrame(inner_mixed_df).copy()

    for column in check_columns: 
        df['equal'] = df[str(column)+'_x'] == df[str(column)+'_y'] # TODO: fix overwriting after each column 
        
        # anticipate that the following attributes from student-por also apply for student-mat
        # df[str(column)+'_y'] = df[str(column)+'_y'].combine_first(df[str(column)+'_x'])

    xy_columns = []
    xy_columns.extend([str(column)+'_x' for column in check_columns])
    xy_columns.extend([str(column)+'_y' for column in check_columns])
    xy_columns.sort()
    return df[xy_columns][df['equal'] == False]

check_contraditories(inner_mixed_df)

In [ ]:
inner_mixed_info_series = pd.DataFrame([['inner_mixed_df', len(inner_mixed_df), len(inner_mixed_df.columns), np.mean(inner_mixed_df.isnull().sum())]], 
                                      columns=['table name', 'table size', 'number of columns', 'average missing values per column'])
left_mixed_info_series = pd.DataFrame([['left_mixed_df', len(left_mixed_df), len(left_mixed_df.columns), np.mean(left_mixed_df.isnull().sum())]], 
                                      columns=['table name', 'table size', 'number of columns', 'average missing values per column'])
info_df = pd.concat([info_df, inner_mixed_info_series, left_mixed_info_series])
info_df

## Data analysis

### Individuelle und strukturelle Faktoren

#### Individuelle Faktoren

In [ ]:
# median dalc / walc per g3 group 
# g3 groups are the median of both g3 points
stacked_bar_df = pd.DataFrame()
stacked_bar_df['g3'] = pd.concat([inner_mixed_df['G3_x'], inner_mixed_df['G3_y']], axis=1).median(axis=1).astype(float)

# check contradictories
result = check_contraditories(inner_mixed_df, ['Dalc'])
print(f"{np.round(len(result) / len(inner_mixed_df),2)} % of contradictory values")
result2 = check_contraditories(inner_mixed_df, ['Walc'])
print(f"{np.round(len(result2) / len(inner_mixed_df),2)} % of contradictory values")
# data filtering
stacked_bar_df['dalc'] = pd.concat([inner_mixed_df['Dalc_x'], inner_mixed_df['Dalc_y']], axis=1).median(axis=1)
stacked_bar_df['walc'] = pd.concat([inner_mixed_df['Walc_x'], inner_mixed_df['Walc_y']], axis=1).median(axis=1)
stacked_bar_df.drop(index=result.index.append(result2.index), inplace=True)

# Define the g3 ranges (bins)
g3_bins = range(2,22,2)

# Create a new column with the bin labels for each g3 value
stacked_bar_df['g3_range'] = pd.cut(stacked_bar_df['g3'], bins=g3_bins)

# Group by the g3_range column and compute the median dalc and walc
stacked_bar_df = stacked_bar_df.groupby('g3_range')[['dalc', 'walc']].median().reset_index()

# Convert the Interval objects in the 'g3_range' column to strings
stacked_bar_df['g3_range'] = stacked_bar_df['g3_range'].astype(str).str[:-1] + ')'
stacked_bar_df

In [ ]:
# Create a stacked bar graph using Plotly
stacked_bar = go.Figure()

# Add the first trace (bar) to the figure
stacked_bar.add_trace(go.Bar(
    x=stacked_bar_df["g3_range"],
    y=stacked_bar_df["dalc"],
    name='Alkoholkonsum an Arbeitstagen',
    marker_color='#B11226'  # Set color to wine
))

# Add the second trace (bar) to the figure
stacked_bar.add_trace(go.Bar(
    x=stacked_bar_df["g3_range"],
    y=stacked_bar_df["walc"],
    name='Alkoholkonsum am Wochenende',
    marker_color='#FBB117'  # Set color to yellow
))

# Configure the layout for a stacked bar graph
stacked_bar.update_layout(
    barmode='stack',
    title='Einfluss von Alkoholkonsum auf die Lernleistung',
    xaxis_title='Median der erreichten Notenpunkte <br>in beiden Fächern',
    yaxis_title='Alkoholkonsumeeinschätzung<br>(1: sehr gering - 5: sehr stark)',
    yaxis = dict(range=[0,10])
)

# Show the plot
stacked_bar.show()

#### Strukturelle Faktoren

In [ ]:
violin_df = pd.DataFrame()

mapping = {1: '<15 min', 2: '15 - 30 min', 3: '30-60 min', 4: '>60 min'}
violin_df['traveltime'] = inner_mixed_df['traveltime_x'].map(mapping)

violin_df['internet'] = inner_mixed_df['internet']
violin_df['g3'] = pd.concat([inner_mixed_df['G3_x'], inner_mixed_df['G3_y']], axis=1).median(axis=1)

In [ ]:
# Define colors for 'internet' categories
colors = {'yes': 'green', 'no': 'red'}

# Create a violin plot
violin = go.Figure()

def translate(x):
    return 'Ja' if x=='yes' else 'Nein'

for internet_status, color in colors.items():
    subset = violin_df[violin_df['internet'] == internet_status]
    side_value = 'positive' if internet_status=='yes' else 'negative'
    violin.add_trace(go.Violin(
        x=subset['traveltime'],
        y=subset['g3'],
        box_visible=False,
        side=side_value,
        meanline_visible=True,
        line_color= 'black',# 'dark'+color,
        fillcolor=color,
        opacity=0.5,
        name=translate(internet_status)
    ))

# Update layout
violin.update_layout(
    title='Zusammenhang struktureller Faktoren und Lernleistungen',
    xaxis_title='Pendelzeit',
    yaxis_title='Median der erreichten Notenpunkte <br>in beiden Fächern',
    xaxis=dict(
        categoryorder='array',
        categoryarray=['<15 min', '15 - 30 min', '30-60 min', '>60 min']
    ),
    annotations=[
        dict(
            x=1.1,
            y=1.15,
            align="right",
            valign="top",
            text='Internet <br> available',
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top"
        )
    ]
)

# Show the plot
violin.show()

### Soziale Faktoren

#### familiäres Umfeld

In [ ]:
bar_df = pd.DataFrame()
bar_df['g3'] = pd.concat([inner_mixed_df['G3_x'], inner_mixed_df['G3_y']], axis=1).median(axis=1)
bar_df['famrel'] = pd.concat([inner_mixed_df['famrel_x'], inner_mixed_df['famrel_y']], axis=1).median(axis=1)
bar_df['famsize'] = inner_mixed_df['famsize']

# result = check_contraditories(inner_mixed_df, ['famrel'])
# print(f"{np.round(len(result) / len(inner_mixed_df),2)} % of contradictory values")
# bar_df.drop(index=result.index, inplace=True)

bar_df

In [ ]:
agg_bar_df = bar_df.groupby(['famsize', 'famrel'])['g3'].median().reset_index()
# Create histograms for each family size
bar = go.Figure()
bar.add_trace(go.Bar(x=agg_bar_df[agg_bar_df["famsize"] == 'LE3']['famrel'], y=agg_bar_df['g3'],name='LE3'))
bar.add_trace(go.Bar(x=agg_bar_df[agg_bar_df["famsize"] == 'GT3']['famrel'], y=agg_bar_df['g3'], name='GT3'))

# Update layout
bar.update_layout(
    title='Einfluss der Familiengröße und dessen <br>Beziehungsverhältnis auf die Lernleistung',
    xaxis_title='Qualität der Familienbeziehung<br>(1: sehr schlecht - 5: sehr gut)',
    yaxis_title='Median der erreichten Notenpunkte <br>in beiden Fächern'
)

# Show the plot
bar.show()


#### partnerliches und freundschaftliches Umfeld

In [ ]:
result = check_contraditories(inner_mixed_df, ['goout'])
print(f"{np.round(len(result) / len(inner_mixed_df),2)} % of contradictory values")
result2 = check_contraditories(inner_mixed_df, ['romantic'])
print(f"{np.round(len(result2) / len(inner_mixed_df),2)} % of contradictory values")

box_df = pd.DataFrame()
box_df['g3'] = pd.concat([inner_mixed_df['G3_x'], inner_mixed_df['G3_y']], axis=1).median(axis=1)
box_df['goout'] = pd.concat([inner_mixed_df['goout_x'], inner_mixed_df['goout_y']], axis=1).median(axis=1)
box_df['romantic'] = inner_mixed_df['romantic_x']
box_df.drop(index=result.index.append(result2.index), inplace=True)
box_df

In [ ]:
box = go.Figure()

for index, (category, color) in enumerate(zip(['no', 'yes'], ['grey', '#E41B17'])):
    df_category = box_df[box_df['romantic'] == category]
    box.add_trace(go.Box(
        x=df_category['goout'],
        y=df_category['g3'],
        name=f'Romantic = {category}',
        marker_color=color,
        offsetgroup=index
    ))

box.update_layout(
    title='Boxplot-Verteilungen zum freundschaftlichen und partnerlichen Einfluss',
    xaxis_title='Intensität sozialer Aktivitäten im Freundeskreis',
    yaxis_title='Median der erreichten Notenpunkte <br>in beiden Fächern',
    boxmode='group'
)

box.show()

### Einfluss der individuellen Leistungsbereitschaft

#### Einfluss von Lernunterstützung

In [ ]:
balk_df = pd.DataFrame()
mat_df['mat_sup_sum'] = (mat_df[['schoolsup', 'famsup', 'paid']] == 'yes').sum(axis=1)
por_df['por_sup_sum'] = (por_df[['schoolsup', 'famsup', 'paid']] == 'yes').sum(axis=1)

aggr_mat_balk_df = mat_df.groupby(['mat_sup_sum'])['G3'].median().reset_index()
aggr_por_balk_df = por_df.groupby(['por_sup_sum'])['G3'].median().reset_index()

print(aggr_mat_balk_df)
print(aggr_por_balk_df)

In [ ]:
balk = go.Figure()
balk = make_subplots(rows=1, cols=2, subplot_titles=('Mathematik', 'Portugiesisch'))

balk.add_trace(go.Box(
    x=mat_df['G3'],
    y=mat_df['mat_sup_sum'],
    name='Math',
    marker_color='blue',
    offsetgroup=0,
    orientation='h'),
    row=1,
    col=1
)

balk.add_trace(go.Box(
    x=mat_df['G3'],
    y=mat_df['mat_sup_sum'],
    name='Portugiesisch',
    marker_color='#FF0000',
    offsetgroup=0,
    orientation='h'),
    row=1,
    col=2
)

balk.update_layout(
    title='Boxplot-Verteilungen zum Lernunterstützungseinfluss auf die Lernleistung',
    xaxis_title='Median der erreichten Notenpunkte <br>in beiden Fächern',
    yaxis_title='Summe der Lernunterstützungen',
    boxmode='group'
)

balk.show()

#### Individuelle Lernbereitschaft

In [ ]:
studytime_map = {1: '<2 hours', 2: '2 to 5 hours', 3: '5 to 10 hours',4: '>10 hours'}

nextbar_df = pd.DataFrame()
nextbar_df['G3'] = mat_df['G3']
nextbar_df['studytime'] = mat_df['studytime']
nextbar_df['absences_bins'] = pd.cut(mat_df['absences'], bins=4).astype(str)
grouped = nextbar_df.groupby(['studytime', 'absences_bins'])['G3'].median().reset_index()
nextbar_df['studytime'] = nextbar_df['studytime'].map(studytime_map)

grouped

In [ ]:
# Creating a figure with subplots for each studytime value
nextbar = go.Figure()

for absence_bin in grouped['absences_bins'].unique():
    filtered_data = grouped[grouped['absences_bins'] == absence_bin]
    nextbar.add_trace(go.Bar(x=filtered_data['studytime'], y=filtered_data['G3'], name=f"Absences in range {absence_bin}"))

# Updating layout
nextbar.update_layout(
    title="Median G3 by Study Time and Absences",
    xaxis_title="Studytime",
    yaxis_title="Median G3",
    barmode='group',
    # Adjust spacing between subplots
    margin=dict(l=20, r=20, t=50, b=50),
    # Show legend
    legend=dict(title="Studytime")
)

# Show the plot
nextbar.show()